In [ ]:
!pip install pandas numpy matplotlib scipy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

# Display settings
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 200)

print("Libraries loaded successfully.")


ModuleNotFoundError: No module named 'pandas'

In [ ]:
csv_path = "/mnt/data/marketing_data.csv"  # change if needed

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"File not found at {csv_path}")

df = pd.read_csv(csv_path, low_memory=False)
print("Shape:", df.shape)
display(df.head())
display(df.info())
display(df.isna().sum().sort_values(ascending=False).head(20))


In [ ]:
# Convert Dt_Customer → datetime, Income → numeric
if 'Dt_Customer' in df.columns:
    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce')
if 'Income' in df.columns:
    df['Income'] = pd.to_numeric(df['Income'], errors='coerce')

display(df[['Dt_Customer', 'Income']].head(10))


In [ ]:
def clean_string_col(s):
    if pd.isna(s):
        return s
    return str(s).strip().title()

for col in ['Education', 'Marital_Status']:
    if col in df.columns:
        df[col] = df[col].apply(clean_string_col)
        print(f"\nCleaned unique values in {col}:")
        display(df[col].value_counts())


In [ ]:
if 'Income' in df.columns:
    overall_median = df['Income'].median()
    group_cols = [c for c in ['Education', 'Marital_Status'] if c in df.columns]

    if group_cols:
        medians = df.groupby(group_cols)['Income'].median().reset_index().rename(columns={'Income': 'MedianIncome'})
        df = df.merge(medians, on=group_cols, how='left')
        df['Income'] = df['Income'].fillna(df['MedianIncome']).fillna(overall_median)
        df.drop(columns=['MedianIncome'], inplace=True)
    else:
        df['Income'] = df['Income'].fillna(overall_median)

print("Missing Income values after imputation:", df['Income'].isna().sum())


In [ ]:
# Total Children
if {'Kidhome', 'Teenhome'}.issubset(df.columns):
    df['TotalChildren'] = df['Kidhome'] + df['Teenhome']

# Age
if 'Year_Birth' in df.columns and 'Dt_Customer' in df.columns:
    df['Age'] = df['Dt_Customer'].dt.year - df['Year_Birth']
elif 'Year_Birth' in df.columns:
    df['Age'] = pd.Timestamp.now().year - df['Year_Birth']

# Total Spending (sum of all 'Mnt' columns)
mnt_cols = [c for c in df.columns if c.startswith('Mnt')]
if mnt_cols:
    df['TotalSpending'] = df[mnt_cols].sum(axis=1)

# Total Purchases
purchase_cols = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
purchase_cols = [c for c in purchase_cols if c in df.columns]
if purchase_cols:
    df['TotalPurchases'] = df[purchase_cols].sum(axis=1)

display(df[['TotalChildren', 'Age', 'TotalSpending', 'TotalPurchases']].head())


In [ ]:
cols_to_plot = ['Income', 'TotalSpending']
cols_to_plot = [c for c in cols_to_plot if c in df.columns]

for col in cols_to_plot:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].hist(df[col].dropna(), bins=40)
    axes[0].set_title(f"Histogram of {col}")
    axes[1].boxplot(df[col].dropna(), vert=False)
    axes[1].set_title(f"Boxplot of {col}")
    plt.tight_layout()
    plt.show()


In [ ]:
def winsorize_series(s, lower_q=0.01, upper_q=0.99):
    low, high = s.quantile(lower_q), s.quantile(upper_q)
    return s.clip(lower=low, upper=high)

for col in ['Income', 'TotalSpending']:
    if col in df.columns:
        df[col + '_winsor'] = winsorize_series(df[col])


In [ ]:
# Ordinal Encoding for Education
if 'Education' in df.columns:
    df['Education_clean'] = df['Education'].replace({'2Nd Cycle':'2n Cycle','2N Cycle':'2n Cycle','Phd':'PhD'})
    mapping = {'Basic': 0, '2n Cycle': 1, 'Graduation': 2, 'Master': 3, 'PhD': 4}
    df['Education_Ordinal'] = df['Education_clean'].map(mapping)

# One-Hot Encoding for Nominals
nominal_cols = [c for c in ['Marital_Status', 'Country', 'Sex'] if c in df.columns]
df = pd.get_dummies(df, columns=nominal_cols, drop_first=False)

print("Encoding complete.")


In [ ]:
numeric_df = df.select_dtypes(include=[np.number])
corr = numeric_df.corr()

fig, ax = plt.subplots(figsize=(10, 8))
cax = ax.imshow(corr, interpolation='nearest', aspect='auto', cmap='coolwarm')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
fig.colorbar(cax)
plt.title("Correlation Heatmap (Numeric Features)")
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import spearmanr, mannwhitneyu

# 1. Older → More store purchases
if {'Age', 'NumStorePurchases'}.issubset(df.columns):
    rho, p = spearmanr(df['Age'], df['NumStorePurchases'])
    print(f"H1: Age vs Store Purchases → rho={rho:.3f}, p={p:.4f}")

# 2. Children → More web purchases
if {'TotalChildren', 'NumWebPurchases'}.issubset(df.columns):
    grp_with = df[df['TotalChildren'] > 0]['NumWebPurchases']
    grp_without = df[df['TotalChildren'] == 0]['NumWebPurchases']
    u, p = mannwhitneyu(grp_with, grp_without, alternative='greater')
    print(f"H2: Children→Web Purchases → U={u:.3f}, p={p:.4f}")

# 3. Store vs Online+Catalog (Cannibalization)
if {'NumStorePurchases', 'NumWebPurchases', 'NumCatalogPurchases'}.issubset(df.columns):
    other = df['NumWebPurchases'] + df['NumCatalogPurchases']
    rho, p = spearmanr(df['NumStorePurchases'], other)
    print(f"H3: Store vs Other Channels → rho={rho:.3f}, p={p:.4f}")

# 4. USA vs Rest (Total Spending)
if {'Country', 'TotalSpending'}.issubset(df.columns):
    usa = df[df['Country'] == 'USA']['TotalSpending']
    rest = df[df['Country'] != 'USA']['TotalSpending']
    u, p = mannwhitneyu(usa, rest, alternative='greater')
    print(f"H4: USA > Rest Spending → U={u:.3f}, p={p:.4f}")


In [ ]:
# Top & Bottom Products by Revenue
mnt_cols = [c for c in df.columns if c.startswith('Mnt')]
if mnt_cols:
    product_sums = df[mnt_cols].sum().sort_values(ascending=False)
    print("Top Products:\n", product_sums.head(5))
    print("\nLowest Products:\n", product_sums.tail(5))
    product_sums.plot(kind='bar', figsize=(10,4), title='Product Revenue')
    plt.show()

# Children vs Spending
if {'TotalChildren', 'TotalSpending'}.issubset(df.columns):
    df.boxplot(column='TotalSpending', by='TotalChildren', figsize=(8,4))
    plt.title("Total Spending by Number of Children")
    plt.suptitle('')
    plt.show()


In [ ]:
print("""
✅ Key Insights:
- Older customers may prefer store purchases (check correlation result).
- Parents tend to make more web purchases (if p < 0.05).
- Negative correlation between store and other channels → possible cannibalization.
- USA customers might spend significantly more than others.

Next Steps:
- Build predictive models for campaign response.
- Perform segmentation (RFM or clustering).
- Design targeted offers to optimize channel use.
""")
